In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [18]:
#Import the sourcing datasets for hyderabad
df_all = pd.read_csv('sourcing_data_bangalore.csv')
df_all.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,zone_name
0,6339fe07ff856f4b4474592b,0.634,88618925e9fffff,12.938844,77.625885,Bangalore,88618925e9fffff,12.938918,77.626281,23927,2,230,Link,02:59,1,20221003,dropped,NaN,zone1
1,6339f56b3abb71427b9f1275,1.187,88618921c7fffff,12.983010,77.761612,Bangalore,88618920b3fffff,12.954065,77.697990,20243,2,200,Link,02:00,1,20221003,dropped,NaN,zone2
2,6339f64082db09299f140909,1.361,88618925e9fffff,12.937722,77.627966,Bangalore,88618925c5fffff,12.934267,77.619576,20616,2,200,Link,02:00,1,20221003,customerCancelled,Change of plans,zone1
3,633a01b482db09299f140ca8,3.109,88618925c3fffff,12.921890,77.610764,Bangalore,88618925c9fffff,12.916622,77.622650,25508,2,245,Link,02:59,1,20221003,dropped,NaN,zone1
4,6339f9e522fc8029057d6d35,2.096,88618924e3fffff,12.900276,77.585075,Bangalore,8861892433fffff,12.901659,77.608818,22149,2,215,Link,02:00,1,20221003,dropped,NaN,zone1


In [19]:
#Create a count column
df_all['count'] = 1

#Create a new column for cancel_reason to identify OCARA, COBRA and COBRM
df_all['new_cancel_reason'] = np.where(df_all['order_status'] == 'customerCancelled', 
    np.where(df_all['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
    np.where(df_all['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

#Create a column to indicate final order status
df_all['final_order_status'] = np.where(df_all['order_status'] == 'customerCancelled', df_all['new_cancel_reason'], df_all['order_status'])
df_all.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,zone_name,count,new_cancel_reason,final_order_status
0,6339fe07ff856f4b4474592b,0.634,88618925e9fffff,12.938844,77.625885,Bangalore,88618925e9fffff,12.938918,77.626281,23927,2,230,Link,02:59,1,20221003,dropped,NaN,zone1,1,no_cancellation,dropped
1,6339f56b3abb71427b9f1275,1.187,88618921c7fffff,12.983010,77.761612,Bangalore,88618920b3fffff,12.954065,77.697990,20243,2,200,Link,02:00,1,20221003,dropped,NaN,zone2,1,no_cancellation,dropped
2,6339f64082db09299f140909,1.361,88618925e9fffff,12.937722,77.627966,Bangalore,88618925c5fffff,12.934267,77.619576,20616,2,200,Link,02:00,1,20221003,customerCancelled,Change of plans,zone1,1,OCARA,OCARA
3,633a01b482db09299f140ca8,3.109,88618925c3fffff,12.921890,77.610764,Bangalore,88618925c9fffff,12.916622,77.622650,25508,2,245,Link,02:59,1,20221003,dropped,NaN,zone1,1,no_cancellation,dropped
4,6339f9e522fc8029057d6d35,2.096,88618924e3fffff,12.900276,77.585075,Bangalore,8861892433fffff,12.901659,77.608818,22149,2,215,Link,02:00,1,20221003,dropped,NaN,zone1,1,no_cancellation,dropped


In [20]:
#Create a list of hexes belonging to both of the batching areas
zone1_hexes = list(df_all[df_all['zone_name'] == 'zone1']['customer_location_hex_8'].unique())
zone2_hexes = list(df_all[df_all['zone_name'] == 'zone2']['customer_location_hex_8'].unique())

#First we will check the captain_hexes for orders that resulted in drop
df_all['captain_customer_same_area'] = np.where(df_all['zone_name'] == 'zone1',
 np.where(df_all['captain_location_hex_8'].isin(zone1_hexes), 'sourced_from_batching_area', 'sourced_from_outside_batching_area'),
 np.where(df_all['captain_location_hex_8'].isin(zone2_hexes), 'sourced_from_batching_area', 'sourced_from_outside_batching_area'))

#Check the number of orders serviced within the batching area vs outside the batching area by zone
sourced_zone = pd.pivot_table(data = df_all, index = ['zone_name', 'yyyymmdd', 'hour', 'final_order_status'], columns = ['captain_customer_same_area'], 
values = 'count', aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Create a column for total rides
sourced_zone['total'] = sourced_zone[['sourced_from_batching_area', 'sourced_from_outside_batching_area']].sum(axis = 1)

#Calculate the percentage of each order_status
sourcing_zone = ['sourced_from_batching_area', 'sourced_from_outside_batching_area']
for zone in sourcing_zone:
  sourced_zone['{}_%'.format(zone)] = round(sourced_zone[zone]/sourced_zone['total'], 4)
sourced_zone.head()

,zone_name,yyyymmdd,hour,final_order_status,sourced_from_batching_area,sourced_from_outside_batching_area,total,sourced_from_batching_area_%,sourced_from_outside_batching_area_%
0,zone1,20220919,0,COBRA,160,3,163,0.9816,0.0184
1,zone1,20220919,0,COBRM,1,0,1,1.0000,0.0000
2,zone1,20220919,0,OCARA,131,12,143,0.9161,0.0839
3,zone1,20220919,0,dropped,50,53,103,0.4854,0.5146
4,zone1,20220919,0,expired,7,5,12,0.5833,0.4167


In [21]:
#Check if the captain was sourced from the exact same hex where the customer location is
df_all['captain_customer_same_hex'] = np.where(df_all['captain_location_hex_8'] == df_all['customer_location_hex_8'], 'sourced_from_same_hex',
 'sourced_from_different_hex')

#Check the number of orders serviced from the same hex
sourced_hex = pd.pivot_table(data = df_all, index = ['zone_name', 'yyyymmdd', 'hour', 'final_order_status'], columns = ['captain_customer_same_hex'], 
values = 'count', aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Create a column for total rides
sourced_hex['total'] = sourced_hex[['sourced_from_same_hex', 'sourced_from_different_hex']].sum(axis = 1)

#Calculate the percentage of each order_status
sourcing_hex = ['sourced_from_same_hex', 'sourced_from_different_hex']
for hex in sourcing_hex:
  sourced_hex['{}_%'.format(hex)] = round(sourced_hex[hex]/sourced_hex['total'], 4)
sourced_hex.head()

,zone_name,yyyymmdd,hour,final_order_status,sourced_from_different_hex,sourced_from_same_hex,total,sourced_from_same_hex_%,sourced_from_different_hex_%
0,zone1,20220919,0,COBRA,6,157,163,0.9632,0.0368
1,zone1,20220919,0,COBRM,0,1,1,1.0000,0.0000
2,zone1,20220919,0,OCARA,86,57,143,0.3986,0.6014
3,zone1,20220919,0,dropped,95,8,103,0.0777,0.9223
4,zone1,20220919,0,expired,12,0,12,0.0000,1.0000


In [22]:
#Check for the combinations for each customer-captain hex
df_dropped = df_all[df_all['final_order_status'] == 'dropped']
hex_combos = df_dropped.groupby(['customer_location_hex_8', 'captain_location_hex_8'])['count'].sum().reset_index()
hex_combos = hex_combos.sort_values(by = ['customer_location_hex_8', 'count'], ascending = False).reset_index(drop = True)

#Get the total orders done by customer  location
customer_orders = df_dropped.groupby(['customer_location_hex_8'])['count'].sum().reset_index()
customer_orders = customer_orders.rename(columns = {'count': 'total'})

#Merge customer orders with hex_combos
hex_combos = hex_combos.merge(customer_orders, on = 'customer_location_hex_8', how = 'left')
hex_combos['%_of_orders'] = round(hex_combos['count']/hex_combos['total'], 4)
hex_combos.head(10)

,customer_location_hex_8,captain_location_hex_8,count,total,%_of_orders
0,8861892e49fffff,88618920a3fffff,381,6153,0.0619
1,8861892e49fffff,88618920a7fffff,362,6153,0.0588
2,8861892e49fffff,8861892e49fffff,283,6153,0.0460
3,8861892e49fffff,88618920bdfffff,268,6153,0.0436
4,8861892e49fffff,8861892e4dfffff,267,6153,0.0434
5,8861892e49fffff,8861892e47fffff,210,6153,0.0341
6,8861892e49fffff,8861892e0dfffff,199,6153,0.0323
7,8861892e49fffff,8861892e4bfffff,189,6153,0.0307
8,8861892e49fffff,886189219bfffff,164,6153,0.0267
9,8861892e49fffff,88618920b1fffff,145,6153,0.0236


In [23]:
#Check the percentile distribution for each type of order per day
percentile_FM = df_all.groupby(['zone_name', 'final_order_status', 'yyyymmdd',
 'hour'])['accept_to_pickup_distance'].describe(percentiles = [0.25, 0.5, 0.75, 0.9, 0.99]).reset_index()
percentile_FM = percentile_FM.fillna(0)
percentile_FM.head()

,zone_name,final_order_status,yyyymmdd,hour,count,mean,std,min,25%,50%,75%,90%,99%,max
0,zone1,COBRA,20220919,0,33.0,1.818727,1.283574,0.105,0.8350,1.5880,2.52000,3.2876,5.68000,6.424
1,zone1,COBRA,20220919,1,18.0,1.388278,0.860164,0.219,0.7640,1.2540,1.96950,2.2762,3.25017,3.386
2,zone1,COBRA,20220919,2,9.0,0.979556,0.665152,0.001,0.6770,1.0020,1.26200,1.6552,2.11312,2.164
3,zone1,COBRA,20220919,3,4.0,0.328750,0.365967,0.002,0.0635,0.2545,0.51975,0.6903,0.79263,0.804
4,zone1,COBRA,20220919,4,5.0,1.914000,1.010243,0.747,0.9680,2.3490,2.41500,2.8206,3.06396,3.091


In [24]:
#Save all the datasets to file
sourced_zone.to_csv('bangalore/sourcing_from_batch.csv', index = False)
sourced_hex.to_csv('bangalore/sourcing_from_hex.csv', index = False)
hex_combos.to_csv('bangalore/hex_combos.csv', index = False)
percentile_FM.to_csv('bangalore/fm_percentile_dist.csv', index = False)